# Inference Pipeline with Custom Containers and xgBoost
Typically a Machine Learning (ML) process consists of few steps: data gathering with various ETL jobs, pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm. 
In many cases, when the trained model is used for processing real time or batch prediction requests, the model receives data in a format which needs to pre-processed (e.g. featurized) before it can be passed to the algorithm. In the following notebook, we will demonstrate how you can build your ML Pipeline leveraging the ability to create custom Sagemaker algorithms and the out of the box SageMaker xgBoost algorithm. After the model is trained we will deploy the ML Pipeline (data preprocessing, the xgBoost classifier, and data postprocessing) as an Inference Pipeline behind a single SageMaker Endpoint for real time inference. We will also use the preprocessor with batch transformation using Amazon SageMaker Batch Transform to prepare xgBoost training data.

![Inference Diagram](./Inference_diagram.png)

The toy problem that is being solved here is to match a set of keywords to a category of questions. From there we can match that category against a list of available agents who specialize in answering that category of question. The agents and their availability is stored externally in a DynamoDB database. The data transformations, matching against our model, and querying of the database are all done as part of the inference pipeline.

The preprocessing step of the pipeline encodes a comma-separated list of words into a format that xgBoost understands using a CountVectorizer. It also trains a LabelEncoder, which is used to transform from the categories of questions to a set of integers - having the labels encoded as integers is also a requirement of the xgBoost multiclass classifer. 

The xgBoost model maps the encoded list of words to an integer, which represents the encoded class of question that best matches those words.

Finally, the postprocessing step of the pipeline uses the LabelEncoding model trained in the preprocessing step to map the number representing the classification of the question back to the text. It then takes the category and queries dynamodb for available agents that matches that category.

Let's first create our Sagemaker session and role, and create a S3 prefix to use for the notebook example.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!mkdir -p returns_data
!python3 generate-training-data.py --samples 100000 --filename returns_data/samples.csv

In [ ]:
!python3 load-ddb-data.py PipelineLookupTable

In [ ]:
# S3 prefix

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = "Custom-Pipeline-Inference-Example"

## Upload the data for training <a class="anchor" id="upload_data"></a>

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. We can use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [ ]:
WORK_DIRECTORY = "returns_data"

train_input = sagemaker_session.upload_data(
    path="{}/{}".format(WORK_DIRECTORY, "samples.csv"),
    bucket=bucket,
    key_prefix="{}/{}".format(prefix, "train"),
)

### Set up a loader function

The load_data function pulls in the CSV data into two columns: the first column of the CSV is mapped to the label, and every subsequent CSV column is loaded as a dictionary into the second Pandas column

In [ ]:
import pandas as pd
import csv


def load_data(raw, columns, skip_first_row=True):
    recs = [(row[0], set(row[1:])) for row in csv.reader(raw)]
    if skip_first_row:
        return pd.DataFrame.from_records(recs[1:], columns=columns)
    else:
        return pd.DataFrame.from_records(recs, columns=columns)


def load(files, columns, skip_first_row=True):
    raw_data = []
    for file in files:
        raw_data.append(load_data(open(file), columns, skip_first_row))

    return pd.concat(raw_data)

In [ ]:
df = load(["returns_data/samples.csv"], ["label", "words"])

In [ ]:
df.head()

In [ ]:
num_labels = len(df["label"].unique())
num_labels

### Words in our dataset

Let's take a look at the set of words being used. We use a CountVectorizer with the set analyzer to encode the column.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer=set)
count_res = vectorizer.fit_transform(df["words"])
vectorizer.get_feature_names_out()

In [ ]:
import boto3

ecr_namespace = "custompipeline/"
prefix = "preprocessor"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
print(container_image_uri)

In [ ]:
import sagemaker

custom_preprocessor = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    # instance_type='local', # use local mode
    instance_type="ml.m5.4xlarge",
    base_job_name=prefix,
)

train_config = sagemaker.inputs.TrainingInput(
    s3_data="s3://{0}/{1}/train/".format(bucket, prefix),
    content_type="text/csv"
)

val_config = sagemaker.inputs.TrainingInput(
    s3_data="s3://{0}/{1}/val/".format(bucket, prefix),
    content_type="text/csv"
)

custom_preprocessor.fit({"train": train_input})

In [ ]:
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_name = "simple-ep-" + timestamp_prefix
predictor = custom_preprocessor.deploy(
    initial_instance_count=1, instance_type="ml.c5.4xlarge", endpoint_name=endpoint_name
)

In [ ]:
from sagemaker.serializers import (
    JSONSerializer,
    CSVSerializer
)
from sagemaker.deserializers import (
    JSONDeserializer
)
from sagemaker.predictor import (
    Predictor
)

payload = "rental,peanut,butter\ncovid"
print("content type csv", "text/csv")

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer()
)

print(predictor.predict(payload))

In [ ]:
sm_client = sagemaker_session.boto_session.client("sagemaker")
sm_client.delete_endpoint(EndpointName=endpoint_name)

## Batch transform our training data <a class="anchor" id="preprocess_train_data"></a>
Now that our proprocessor is properly fitted, let's go ahead and preprocess our training data. Let's use batch transform to directly preprocess the raw data and store right back into s3.

In [ ]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = custom_preprocessor.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", assemble_with="Line", accept="text/csv"
)

In [ ]:
# Preprocess training input
transformer.transform(train_input, content_type="text/csv", split_type="Line")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path
print(preprocessed_train)

## Fit a xgBoost Model with the preprocessed data <a class="anchor" id="training_model"></a>
Let's take the preprocessed training data and fit a xgBoost Model. Sagemaker provides prebuilt algorithm containers that can be used with the Python SDK.

In [ ]:
xgboost_container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')
xgboost_container

In [ ]:
xgboost_hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "silent": "0",
    "objective": "multi:softmax",
    "num_class": num_labels,
    "num_round": "10",
}

# set an output path where the trained model will be saved
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "xgboost-pipeline-training")

In [ ]:
xgboost_estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    hyperparameters=xgboost_hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.4xlarge",
    volume_size=5,  # 5 GB
    output_path=output_path,
)

In [ ]:
xgboost_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)

In [ ]:
# execute the XGBoost training job
xgboost_estimator.fit({"train": xgboost_train_data})

# Serial Inference Pipeline with the preprocessor, xgBoost classifier and postprocessor <a class="anchor" id="serial_inference"></a>


## Set up the inference pipeline <a class="anchor" id="pipeline_setup"></a>
Setting up a Machine Learning pipeline can be done with the Pipeline Model. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted preprocessor model, the fitted xgBoost model, and the postprocessor (which uses the preprocessor model data). Deploying the model follows the same ```deploy``` pattern in the SDK.

Notice that we pass in the trained model from the proprocessor into the postpostprocessor. The reason we do this is so that the postprocesser can access the LabelEncoder that was trained in the preprocessor to invert that operation. This allows the inference pipeline to return the actual name of the category instead of the category label (e.g. "medical" instead of 7).

In [ ]:
ecr_namespace = "custompipeline/"
prefix = "postprocessor"
ecr_repository_name = ecr_namespace + prefix
postprocessor_container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
postprocessor_container_image_uri

In [ ]:
from sagemaker.model import Model

# print(Model.__init__.__doc__)
custom_postprocessor = Model(
    image_uri=postprocessor_container_image_uri,
    model_data=custom_preprocessor.model_data,
    role=role,
    sagemaker_session=sagemaker_session,
)

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

preprocessor_model = custom_preprocessor.create_model()
classifier_model = xgboost_estimator.create_model()
postprocessor_model = custom_postprocessor

model_name = "inference-pipeline-" + timestamp_prefix
endpoint_name = "inference-pipeline-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[preprocessor_model, classifier_model, postprocessor_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.c4.xlarge", endpoint_name=endpoint_name)

## Make a request to our pipeline endpoint <a class="anchor" id="pipeline_inference_request"></a>

Here we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). The ```ContentType``` field configures the first container, while the ```Accept``` field configures the last container. You can also specify each container's ```Accept``` and ```ContentType``` values using environment variables.


In [ ]:
from sagemaker.serializers import (
    JSONSerializer,
    CSVSerializer
)
from sagemaker.deserializers import (
    JSONDeserializer
)
from sagemaker.predictor import (
    Predictor
)

payload = "rental,peanut,butter\ncovid"

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer()
)

print(predictor.predict(payload, initial_args={"Accept": "application/json"}))

## Delete Endpoint <a class="anchor" id="delete_endpoint"></a>
Once we are finished with the endpoint, we clean up the resources!

In [ ]:
sm_client = sagemaker_session.boto_session.client("sagemaker")
sm_client.delete_endpoint(EndpointName=endpoint_name)